In [1]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../../python-tools'))
	print(os.getcwd())
except:
	pass


  ### 1. Timing

  * How long does it take for a function to run?

  ```
  * The inner function (“wrapper”) will run the original function
  * But it’ll keep track of the time before and after doing so.
  * Before returning the result to the user, write the timing information to a logfile
  ```

In [2]:
import time
import functools



In [3]:
def logtime(func):

    def wrapper(*args, **kwargs):

        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time

        with open('timelog.txt', 'a') as outfile:
            outfile.write(f'{time.time()}\t{func.__name__}\t{total_time}\n')

        return result

    return wrapper




In [4]:
@logtime

def slow_add(a, b):
    time.sleep(2)
    return a + b




In [5]:
@logtime

def slow_mul(a, b):
    time.sleep(3)
    return a * b




In [6]:
# test function
slow_add(100,100)


200

  records from timelogs:

  1565464610.880044	slow_add	2.0048961639404297


In [7]:
# alternative method
def timethis(func):
    '''
    decortor that reports the execution time.
    '''
    @functools.wraps(func)  
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper



In [8]:
@timethis
def countdown(n):
    '''
    counts down
    '''
    while n > 0:
        n -= 1



In [9]:
countdown(10000)


countdown 0.0009963512420654297


  ### 2. Run function once per minute

  * Raise an exception if we try to run a function more than once in 60 seconds

In [10]:
def once_per_minute(func):

    last_invoked = 0

    def wrapper(*args, **kwargs):

        nonlocal last_invoked

        elapsed_time = time.time() - last_invoked

        if elapsed_time < 60:

            raise RuntimeError(f"Only {elapsed_time} has passed")

        last_invoked = time.time()

        return func(*args, **kwargs)

    return wrapper




In [11]:
@once_per_minute

def add(a, b):
    return a + b




In [12]:
add(2,3)



5

In [13]:
#  add(2,3)

#  ---------------------------------------------------------------------------
#  RuntimeError                              Traceback (most recent call last)
#   in
#  ----> 1 add(2,3)
#   in wrapper(*args, **kwargs)
#       11         if elapsed_time < 60:
#       12
#  ---> 13             raise RuntimeError(f"Only {elapsed_time} has passed")
#       14
#       15         last_invoked = time.time()
#  RuntimeError: Only 10.496497869491577 has passed


  ### 3. Perform function per N seconds
  * Raise an exception if we try to run a function more than once in n seconds

In [14]:
def once_per_n(n):

    def middle(func):
        last_invoked = 0

        def wrapper(*args, **kwargs):
            nonlocal last_invoked
            elapsed_time = time.time() - last_invoked
            if elapsed_time < n:
                raise RuntimeError(f"Only {elapsed_time} has passed")

            last_invoked = time.time()
            return func(*args, **kwargs)

        return wrapper

    return middle




In [15]:
@once_per_n(10)

def add(a, b):
    return a + b




In [16]:
add(2,3)



5

In [17]:
#  add(2,3)

#  ---------------------------------------------------------------------------
#  RuntimeError                              Traceback (most recent call last)
#   in
#  ----> 1 add(2,3)
#   in wrapper(*args, **kwargs)
#        8             elapsed_time = time.time() - last_invoked
#        9             if elapsed_time < n:
#  ---> 10                 raise RuntimeError(f"Only {elapsed_time} has passed")
#       11
#       12             last_invoked = time.time()
#  RuntimeError: Only 0.025335073471069336 has passed
#  

  ### 4. Memoization
  * Cache the results of function calls, so we don’t need to call them again

In [18]:
import pickle




In [19]:
# Executes each time the decorated function is executed
def memoize(func):
    cache = {}

    # Executes each time the decorated function is executed
    def wrapper(*args, **kwargs):
        t = (pickle.dumps(args), pickle.dumps(kwargs))
        if t not in cache:
            print(f"Caching NEW value for {func.__name__}{args}")
            cache[t] = func(*args, **kwargs)
        else:
            print(f"Using OLD value for {func.__name__}{args}")

        return cache[t]

    return wrapper




In [20]:
@memoize
def add(a, b):
    print("Running add!")
    return a + b




In [21]:
@memoize
def mul(a, b):
    print("Running mul!")
    return a * b




In [22]:
add(2,2)




Caching NEW value for add(2, 2)
Running add!


4

In [23]:
add(2,2)


Using OLD value for add(2, 2)


4

  ### 5. Attributes
  * Give many objects the same attributes, but without using inheritance

In [24]:
def fancy_repr(self):
    return f"I'm a {type(self)}, with vars {vars(self)}"




In [25]:
def better_repr(c):
    c.__repr__ = fancy_repr
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        return o
    return wrapper




In [26]:
# Alternative approach
def better_repr(c): #The decorated class
    c.__repr__ = fancy_repr
    return c #Return a callable — class




In [27]:
@better_repr
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y




In [28]:
f = Foo(10, [10, 20, 30])
print(f)


I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30]}


  * The **@object_birthday** decorator, when applied to a class, will add a new **created_at** attribute to new objects
  * Give every object its own birthday. This will contain the timestamp at which each instance was created

In [29]:
def object_birthday(c): #The decorated class
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper




In [30]:
@object_birthday
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y




In [31]:
f = Foo(10, [10, 20, 30])
print(f)
print(f._created_at)




1569963587.2473645


In [32]:
def object_birthday(c):
    c.__repr__ = fancy_repr  #Add a method to the class

    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()  #Add an attribute to the instance
        return o
    return wrapper




In [33]:
@object_birthday
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y





In [34]:
f = Foo(10, [10, 20, 30])
print(f)



I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30], '_created_at': 1569963628.7487805}


 ### 6. Unwrapping a decorator
 * gain access to the original function by accessing the `__wrapped__` attribute.

In [35]:
# decorator has been implemented properly using @wraps
@timethis
def add(x,y):
    return x+y

original_add = add.__wrapped__
original_add(3,4)



7

 ### 7. Preserving function metadata
 * apply the @wraps decorator from functool library to the underlying wrapper function.

In [36]:
def timethis(func):
    '''
    decorator that reports the execution time.
    '''
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


In [37]:
@timethis
def countdown(n:int):
    '''
    counts down
    '''
    while n>0:
        n-=1


In [38]:
countdown(100000)

countdown 0.006980419158935547


In [39]:
countdown.__name__

'countdown'

In [40]:
countdown.__doc__


'\n    counts down\n    '

In [41]:
countdown.__annotations__


{'n': int}

 ### 8. decorator that takes arguments

In [42]:
import logging

def logged(level, name = None, message = None):
    '''
    add logging to a function.
    level is the logging level, name is the logger name, and message is the log message. If name and message aren't specified, they default to the function's module and name
    '''
    def decorate(func):
        logname = name if name else func.__name__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [43]:
# example usage
@logged(logging.DEBUG)
def add(x, y):
    return x+y


In [44]:
@logged(logging.CRITICAL)
def spam():
    print('Spam')


 ### 8. Type checking on function
 * use inspect.signature() function

In [45]:
import inspect


In [46]:
def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # if in optimized mode, disable type checking
        if not __debug__:
            return func
        
        # map function argument names to supplied types
        sig = inspect.signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                    'Argument {} must be {}'.format(name, bound_types[name]))
            
            return func(*args, **kwargs)
        return wrapper
    return decorate




In [47]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x,y,z)



In [48]:
spam(1,2,3)


1 2 3


In [49]:
spam(1, 'hello', 3)


1 hello 3


In [50]:
# spam(1,'hello','world')
# ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
#  in ()
# ----> 1 spam(1,'hello','world')

#  in wrapper(*args, **kwargs)
#      17                     if not isinstance(value, bound_types[name]):
#      18                         raise TypeError(
# ---> 19                     'Argument {} must be {}'.format(name, bound_types[name]))
#      20 
#      21             return func(*args, **kwargs)

# TypeError: Argument z must be 

In [51]:
def spam(x,y,z=42):
    pass

In [52]:
sig = inspect.signature(spam)
print(sig)

(x, y, z=42)


In [53]:
sig.parameters


mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [54]:
sig.parameters['z']._default


42

In [55]:
sig.parameters['z'].kind



<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [56]:
bound_types = sig.bind_partial(int, z = int)
bound_types


<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [57]:
bound_types.arguments


OrderedDict([('x', int), ('z', int)])

In [58]:
bound_values = sig.bind(1,2,3)
bound_values.arguments

OrderedDict([('x', 1), ('y', 2), ('z', 3)])